# CS295/395: Secure Distributed Computation
## Homework 7

In [1]:
# Imports and definitions
import numpy as np
import galois
import random
bits = 128

from collections import defaultdict
from collections import namedtuple

## Definitions: Paillier Cryptosystem

In [2]:
def keygen(bits):
    """Generates keys with `bits`-bits of security. Returns a pair: (secret key, public key)."""
    def invmod(x, m):
        gcd, s, t = galois.egcd(x, m)
        assert gcd == 1
        return s

    p = galois.random_prime(int(bits/2))
    q = galois.random_prime(int(bits/2))

    n = p*q
    g = n+1
    lamb = (p-1) * (q-1)
    mu = invmod(lamb, n)
    
    sk = (lamb, mu)
    pk = (n, g)
    return sk, pk

def encrypt(m, pk):
    """Encrypts the message `m` with public key `pk`."""
    n, g = pk
    n_sq = n**2
    r = random.randint(1, n)
    c = (pow(g, m, n_sq) * pow(r, n, n_sq)) % n_sq
    return c

def decrypt(c, sk, pk):
    """Decrypts the ciphertext `c` using secret key `sk` and public key `pk`."""
    lamb, mu = sk
    n, g = pk
    n_sq = n**2
    L_result = (pow(c, lamb, n_sq) - 1)//n
    return (L_result * mu) % n

def e_add(c1, c2, pk):
    """Add one encrypted integer to another"""
    n, g = pk

    return c1 * c2 % n**2

## Question 1 (30 points)

Implement an electronic voting system using the Paillier cryptosystem. The system has two parts: a voter encrypts their vote using the cryptosystem and submits it to the server, and the server stores the encrypted votes and tallies them at the end of the election. The election server holds the public and private keys; the public key is available to the voters, so that they can encrypt their votes. Each voter should be able to verify that the server has correctly stored their vote, by consulting the encrypted votes to check that their vote indeed appears. Generate your public and private keys with at least 32-bit security.

*Reference:* [Prêt à Voter with Paillier encryption](https://dl.acm.org/doi/10.1016/j.mcm.2008.05.015).

In [14]:
voting_table = ['Candidate 1',
                'Candidate 2',
                'Candidate 3']

class Voter:
    def vote(self, candidate, server):
        """Submits an encrypted vote to the server"""
        pk = server.get_public_key()
        vote_vector = [0] * len(voting_table)
        vote_vector[voting_table.index(candidate)] = 1

        server.submit_vote(list(map(lambda x: encrypt(x, pk), vote_vector)))

class ElectionServer:
    def __init__(self):
        self.votes = []
        self.sk, self.pk = keygen(32)
    
    def get_public_key(self):
        """Get the public key from the election server"""
        return self.pk
    
    def submit_vote(self, ct_vote_vector):
        """Submit an (encrypted) vote to the election server"""
        self.votes.append(ct_vote_vector)
    
    def show_votes(self):
        """Show the submitted (encrypted) votes"""
        return self.votes
    
    def tally_votes(self):
        """Tally up the votes at the end of the election. Returns a list of totals in the 
        same order as the list of candidates."""
        counts = [0] * len(voting_table)
        for vote in self.votes:
            for i in range(len(vote)):
                counts[i] += decrypt(vote[i], self.sk, self.pk)

        return counts

In [17]:
# TEST CASE
es = ElectionServer()

Voter().vote('Candidate 1', es)
Voter().vote('Candidate 2', es)
Voter().vote('Candidate 2', es)
Voter().vote('Candidate 3', es)

print(es.show_votes())
assert es.tally_votes() == [1, 2, 1]

[[20322011918549868218, 46905298359022256488, 65431666188464724967], [75738660543349761488, 60394294651941136084, 73665571411893016204], [50888446385809960771, 39476073820256226535, 52495054763526734813], [10831603828076707535, 43925675549136513615, 78271689507894561167]]


## Question 2 (10 points)

In 2-5 sentences, answer the following:

- What trust assumptions do we make about the *election server* in this election system?
- What trust assumptions do we make about the *voter* in this election system?

- What trust assumptions do we make about the *election server* in this election system?

We are assuming the election server is going to show everyone the same voting billboard, not a fabricated billboard where your vote is real but the others are fake. Otherwise it could show each user their vote in the billboard, but all the others are just votes it creates to promote a certain candidate. We also assume the election server is not going to keep track of the order that people voted and not decrypt the votes until the very end. Otherwise, the votes are not private to the voting server. We are also assuming that it adds up the votes truthfully, because the voter cannot decrypt the votes on the billboard to verify that the server did the addition truthfully.

- What trust assumptions do we make about the *voter* in this election system?

We are assuming that they are going to construct the vote vector appropriately. In this current scheme they can send in vectors like `[0,2,0]` to vote for the same candidate twice or even `[0,-1,0]` to remove a vote from a candidate. We are also assuming that they will not collude with the other voters. If you knew the votes of all but one voter, then you can find out the vote of the non-colluding voter.

## Question 3 (10 points)

In 2-5 sentences, answer the following:

- What is one way a malicious *election server* could break the rules of the election?
- What are two ways a malicious *voter* could break the rules of the election?

- What is one way a malicious *election server* could break the rules of the election?

They could display the encypted votes and usual but then produce a tally that was innacurate. If you voted `[1,0,0]` and the final tally is `[a,b,c]` with $a >= 1$ you cannot go back and "check the servers work" to see if it added truthfully.

- What are two ways a malicious *voter* could break the rules of the election?

They could vote with a vector that looked like `[2,0,-1]` to vote twice for one candidate and unvote the third candidate. They could also collude with the other voters to uncover the vote of a final, non-colluding voter.

## Question 4 (30 points)

The [Boston Women's Workforce Council Gender Pay Gap Survey](https://thebwwc.org/mpc) uses MPC to deploy a survey of Boston-area businesses to determine how women and men are paid differently. Each business submits encrypted values for their employees' salaries, and the system calculates the average salaries for women and men across all of the businesses. This design protects the privacy of individual employees, and protects individual businesses from embarrassment.

Implement a system for conducting a survey like this using the Paillier cryptosystem. Participants should submit their own salaries, and specify their gender. The survey server should collect responses, and at the end of the survey, calculate the average salaries for men and women. Use at least 32-bit security.

In [26]:
class GenderPayGapSurveyParticipant:
    def submit_salary(self, salary, gender, server):
        """Submits an encrypted survey response to the server"""

        salary_vector = [0,0]
        gender_vector = [0,0]

        if gender == "Female":
            salary_vector[0] = salary
            gender_vector[0] = 1
        else:
            salary_vector[1] = salary
            gender_vector[1] = 1

        pk = server.get_public_key()
        server.submit_salary(
            list(map(lambda x: encrypt(x, pk), salary_vector)),
            list(map(lambda x: encrypt(x, pk), gender_vector))
        )

class GenderPayGapSurveyServer:
    def __init__(self):
        self.salaries = []
        self.genders = []
        self.sk, self.pk = keygen(32)
    
    def get_public_key(self):
        return self.pk
        
    def submit_salary(self, ct_salary_vector, ct_gender_vector):
        """Store an entry in the survey"""

        self.salaries.append(ct_salary_vector)
        self.genders.append(ct_gender_vector)
    
    def show_salaries(self):
        """Display the (encrypted) submitted salaries"""
        return self.salaries
    
    def compute_average_salaries(self):
        """Tally the results, decrypt, and return a 2-tuple: (average female salary, average male salary)"""
        
        salary_totals = [0,0]
        
        for salary in self.salaries:
            for i in range(len(salary)):
                salary_totals[i] += decrypt(salary[i], self.sk, self.pk)

        gender_totals = [0,0]

        for gender in self.genders:
            for i in range(len(gender)):
                gender_totals[i] += decrypt(gender[i], self.sk, self.pk)

        return tuple(map(lambda x: x[0]/x[1], zip(salary_totals, gender_totals)))


In [28]:
# TEST CASE
s = GenderPayGapSurveyServer()
GenderPayGapSurveyParticipant().submit_salary(10000, 'Male', s)
GenderPayGapSurveyParticipant().submit_salary(30000, 'Female', s)
GenderPayGapSurveyParticipant().submit_salary(15000, 'Male', s)
GenderPayGapSurveyParticipant().submit_salary(20000, 'Female', s)

assert s.compute_average_salaries() == (25000.0, 12500.0)

## Question 5 (10 points)

In 2-5 sentences, answer the following:

- What is one way a malicious *survey server* could break the rules of the survey?
- What are two ways a malicious *survey participant* could break the rules of the survey?

A malicious servey server could break the rules by showing the correct encrypted salaries table, but then adding up the salaries to anything that they want. The participant has no way to check its work.

The participant could lie about their salary or their gender. They could also submit salaries for multiple genders, or submit a salary with an empty gender vector to increase the total salary amount but not increase the number of participants of that gender.